In [1]:
pip install torch onnx transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 11.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-ma

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Gantilah model_path dengan nama model Anda yang sesuai
model_name = "Equinox391/Athena-Llama-2-7b-chat-finetune"

# Memuat tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Simpan model ke format PyTorch bin
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

In [ ]:
import torch
import onnx
from transformers import AutoModelForCausalLM, AutoTokenizer

# Memuat kembali model dan tokenizer yang sudah disimpan
model = AutoModelForCausalLM.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

# Mendefinisikan input dummy
dummy_input = tokenizer("This is a dummy input", return_tensors="pt").input_ids

# Fungsi ekspor model ke ONNX
torch.onnx.export(
    model,                          # Model yang akan diekspor
    args=(dummy_input,),            # Input dummy
    f="model.onnx",                 # Nama file ONNX yang akan disimpan
    input_names=["input_ids"],      # Nama input dalam model
    output_names=["logits"],        # Nama output dalam model
    opset_version=11,               # Versi opset ONNX
    do_constant_folding=True,       # Melakukan constant folding untuk mengoptimalkan
    use_external_data_format=True,  # Untuk model besar, gunakan format data eksternal
)

print("Model telah berhasil diekspor ke format ONNX.")


In [ ]:
import onnx

# Memuat model ONNX
onnx_model = onnx.load("model.onnx")

# Memverifikasi model
onnx.checker.check_model(onnx_model)
print("Model ONNX valid dan siap digunakan.")
